In [21]:
#import Libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
import seaborn as sns


In [2]:
#Reade data and replace ? with null
data = pd.read_csv("/Users/chris.freyder/stats/case_study/risk_factors_cervical_cancer.csv")
data.dtypes
data.describe
data = data.replace('?', np.nan)
data.isnull().sum()


Age                                     0
Number of sexual partners              26
First sexual intercourse                7
Num of pregnancies                     56
Smokes                                 13
Smokes (years)                         13
Smokes (packs/year)                    13
Hormonal Contraceptives               108
Hormonal Contraceptives (years)       108
IUD                                   117
IUD (years)                           117
STDs                                  105
STDs (number)                         105
STDs:condylomatosis                   105
STDs:cervical condylomatosis          105
STDs:vaginal condylomatosis           105
STDs:vulvo-perineal condylomatosis    105
STDs:syphilis                         105
STDs:pelvic inflammatory disease      105
STDs:genital herpes                   105
STDs:molluscum contagiosum            105
STDs:AIDS                             105
STDs:HIV                              105
STDs:Hepatitis B                  

In [15]:
#Get datasets ready, investigate collinearity
x = data.drop(['Hinselmann', 'Schiller', 'Citology', 'Biopsy'], axis = 1)
y = data[['Biopsy']]


In [27]:

imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imp.fit(x)
transformed_X = imp.transform(x)
# transformed_y = pd.Series(encoder.fit_transform(y))
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, transformed_X, y.values.ravel(), cv=5)

In [28]:
scores

array([0.88372093, 0.93023256, 0.93604651, 0.92982456, 0.9005848 ])